# Imports

In [16]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os 
import pandas as pd
from torchvision.datasets.folder import default_loader
from torchvision.datasets.utils import download_url
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from collections import OrderedDict

from datetime import datetime

# Device

In [17]:
torch.cuda.is_available()

True

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
torch.cuda.device_count()

5

# Dynamic Convolution

# Cifar - 10

In [26]:
classes = 10 
trainset_cifar10 = torchvision.datasets.CIFAR10(root = '/home/varshittha/dynamic-convolution',train=True,
                                                download=True,
                                                transform=transforms.Compose([
                                                transforms.Pad(4),
                                                transforms.RandomCrop(32),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor(),
                                                transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))
                                            ]))
trainloader_cifar10 = torch.utils.data.DataLoader(trainset_cifar10, batch_size=128, shuffle=True, num_workers=0)

testset_cifar10 = torchvision.datasets.CIFAR10(root='/home/varshittha/dynamic-convolution', train=False, download=True,
                                           transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                           ]))
testloader_cifar10 = torch.utils.data.DataLoader(testset_cifar10, batch_size=128, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


# dynamic convolution

# dy_resnet18

##### conv1x1 - dynamic convolution

In [34]:
def conv1x1(in_planes,out_planes,stride = 1): 
    return Dynamic_conv2d(in_planes,out_planes,kernel_size = 1,stride = stride,bias = False)

##### conv3x3 - dynamic convolution

In [35]:
def conv3x3(in_planes,out_planes,stride = 1,groups = 1,dilation = 1): # conv3x3 for dynamic convolution
    return Dynamic_conv2d(in_planes,out_planes,kernel_size = 3,stride = stride,padding = dilation,groups = groups,bias = False,dilation = dilation)

##### BasicBlock

In [36]:
class BasicBlock(nn.Module): # expansion = 1, dilation = 1 , base_width = 64 ,groups = 1
    expansion = 1
    
    def __init__(self,in_planes,out_planes,stride = 1,downsample = None,
                 groups = 1,base_width = 64,dilation = 1,norm_layer = None):
        
        super(BasicBlock,self).__init__()
        
        if base_width != 64:
            raise ValueError('BasicBlock supports only base_width = 64')
        if groups != 1: 
            raise ValueError('BasicBlock supports only groups = 1')
        if dilation > 1:
            raise NotImplementedError('BasicBlock doesnot support dilation > 1')
        # self.conv1 and self.downsample layers downsample the input when stride != 1
        
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        
        self.conv1 = conv3x3(in_planes,out_planes,stride)
        self.bn1   = norm_layer(out_planes)
        self.relu  = nn.RELU(inplace = True) # modify input directly.
        self.conv2 = conv3x3(out_planes,out_planes)
        self.bn2   = norm_layer(out_planes)
        self.downsample = downsample
        self.stride = stride
    
    def forward(self , z):
        
        identity = z
        
        out = self.conv1(z)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(z)
        
        out = out + identity
        out = self.relu(out)
        
        return out
        
        
    

##### Bottleneck

In [ ]:
class Bottleneck(nn.Module):
    
    expansion = 4
    
    def __init__(self,in_planes,out_planes,stride = 1,downsample = None,
                groups = 1,base_width = 64,dilation = 1,norm_layer = None):
        super(Bottleneck,self).__init__()
        

# Model

In [27]:
model =dy_resnet18(num_classes = classes)

NameError: name 'dy_resnet18' is not defined